In [1]:
import os
import pandas as pd
from modules.data_pipeline import read_path, get_path, export_file

export_file.export_to_csv(99, 'igv')
# read_path.read(97, 'igv')

Before filter: (12800, 28)
After filter (5723, 21)
Now processing ---> ('CK', 'ck20240408', 'V001_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (6377, 21)
Now processing ---> ('CK', 'ck20240408', 'V002_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (3258, 21)
Now processing ---> ('CK', 'ck20240408', 'V003_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (3563, 21)
Now processing ---> ('CK', 'ck20240408', 'V004_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (3956, 21)
Now processing ---> ('CK', 'ck20240408', 'V005_IGVData_KPI.csv')
Before filter: (11802, 28)
After filter (4620, 21)
Now processing ---> ('CK', 'ck20240408', 'V006_IGVData_KPI.csv')
Before filter: (12781, 28)
After filter (3228, 21)
Now processing ---> ('CK', 'ck20240408', 'V008_IGVData_KPI.csv')
Before filter: (12800, 28)
After filter (2800, 21)
Now processing ---> ('CK', 'ck20240408', 'V009_IGVData_KPI.csv')
Before filter: (12553, 28)
After filter (3904, 21)
Now processing ---> (

In [ ]:
p = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TJ.csv'
p = os.path.abspath(p)

df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)
df.columns

In [ ]:
def lmd_get_current_task_tag(current_task=str, mission_type=str, location_ref=str):
    if (current_task != 'DSCH') & (current_task != 'LOAD'):
        if ('YARD' in location_ref and mission_type=='DELIVER') or ('QC' in location_ref and mission_type=='RECEIVE'):
            return 'DSCH'
        elif('QC' in location_ref and mission_type=='DELIVER') or ('YARD' in location_ref and mission_type=='RECEIVE'):
            return 'LOAD'      
    else:
        return current_task
    
df['current_task_tag'] = df.apply(
                    lambda x: lmd_get_current_task_tag(x['current_task'], x['mission_type'], x['location_ref']),
                    axis=1)


df.columns

# Test all weekly file

In [ ]:
def get_all_weekly_file():
    root = 'c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99'
    pathlist = []
    for item in os.listdir(root):
        p = os.path.join(root, item)
        if os.path.isfile(p):
            pathlist.append(p)
    return pathlist

li = get_all_weekly_file()

In [ ]:
for p in li:
    df = pd.read_csv(p, encoding='utf-8-sig', index_col=0)
    print(p)
    ins_col = ['location_ref', 'location_tag']
    if ('location_ref' in df.columns) & ('location_tag' in df.columns):
        print(df.columns)
    # else:

    print()

In [4]:
path = os.path.abspath('c:\\Users\westwell\Documents\.WorkDocuments\github\westwell_da\data\processed\W99\Igv_W99_TJ.csv')

d = pd.read_csv(path, encoding='utf-8-sig')
print(d['vesselVisitID'].value_counts())

vesselVisitID
HANB3_003    354607
FULD_000     106592
Name: count, dtype: int64


In [3]:
for cycle, data in d.groupby('Cycle Tag'):
    print(cycle)
    
    # print(data['mission_type'].value_counts())

    print()

A001-C12.0-2024-04-10 02:30:17

A001-C14.0-2024-04-10 02:30:17

A001-C15.0-2024-04-10 02:30:17

A001-C16.0-2024-04-10 02:30:17

A001-C17.0-2024-04-10 02:30:17

A001-C23.0-2024-04-10 02:30:17

A001-C25.0-2024-04-10 02:30:17

A001-C3.0-2024-04-10 02:30:17

A001-C9.0-2024-04-10 02:30:17

A001-FULD_000C1.0-2024-04-10 02:30:17

A001-FULD_000C10.0-2024-04-10 02:30:17

A001-FULD_000C11.0-2024-04-10 02:30:17

A001-FULD_000C12.0-2024-04-10 02:30:17

A001-FULD_000C13.0-2024-04-10 02:30:17

A001-FULD_000C14.0-2024-04-10 02:30:17

A001-FULD_000C15.0-2024-04-10 02:30:17

A001-FULD_000C16.0-2024-04-10 02:30:17

A001-FULD_000C17.0-2024-04-10 02:30:17

A001-FULD_000C18.0-2024-04-10 02:30:17

A001-FULD_000C19.0-2024-04-10 02:30:17

A001-FULD_000C2.0-2024-04-10 02:30:17

A001-FULD_000C20.0-2024-04-10 02:30:17

A001-FULD_000C21.0-2024-04-10 02:30:17

A001-FULD_000C22.0-2024-04-10 02:30:17

A001-FULD_000C23.0-2024-04-10 02:30:17

A001-FULD_000C24.0-2024-04-10 02:30:17

A001-FULD_000C25.0-2024-04-10 02:30: